In [1]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing
from refinitiv.data.content.historical_pricing import Intervals
from refinitiv.data.content.historical_pricing import Adjustments

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [4]:
start_date = "2010-01-01"
end_date = "2024-12-31"

In [6]:
rd.open_session();

In [7]:
response_BTC = historical_pricing.summaries.Definition(
            "BTC="+"CCCL",
            interval=Intervals.DAILY,
            start=start_date,
            end=end_date,
            adjustments = [
                Adjustments.EXCHANGE_CORRECTION,
                Adjustments.MANUAL_CORRECTION
            ],
        ).get_data()

response_ETH = historical_pricing.summaries.Definition(
            "ETH="+"CCCL",
            interval=Intervals.DAILY,
            start=start_date,
            end=end_date,
            adjustments = [
                Adjustments.EXCHANGE_CORRECTION,
                Adjustments.MANUAL_CORRECTION
            ],
        ).get_data()

In [89]:
price_BTC = response_BTC.data.df.TRDPRC_1.dropna()
price_ETH = response_ETH.data.df.TRDPRC_1.dropna()

dates = np.intersect1d(price_BTC.index, price_ETH.index)
dates = dates[dates >= pd.Timestamp('2016-01-01')]

price_BTC = price_BTC.loc[dates]
price_ETH = price_ETH.loc[dates]

returns_BTC = price_BTC.pct_change(fill_method=None).shift(-1).iloc[:-1]
returns_ETH = price_ETH.pct_change(fill_method=None).shift(-1).iloc[:-1]

In [91]:
# save to data folder
price_BTC.to_csv('../data/BTC_price.csv')
price_ETH.to_csv('../data/ETH_price.csv')
returns_BTC.to_csv('../data/BTC_returns.csv')
returns_ETH.to_csv('../data/ETH_returns.csv')

In [96]:
# import sp500 from yahoo finance
import yfinance as yf
sp500 = yf.download('^GSPC', start='2016-01-01')['Adj Close']
sp500_returns = sp500.pct_change(fill_method=None).shift(-1).iloc[:-1]
sp500_returns.to_csv('../data/sp500_returns.csv')

[*********************100%***********************]  1 of 1 completed
